# Quantifying Greenwashing Through Corporate Financial Filings

This project aims to shed light on increasingly common practice of greenwashing, where companies exaggerate the good they’re doing for the world via their ESG environmental, social and governance (ESG) disclosures to entice investors, hoodwink customers and generally improve their financial performance. More specifically, it will look at a) what factors correlate to higher third-party ESG scores and b) the differences between those factors when looking at different ESG ratings.

## About the data

### A) Financial data

Financial data — quarterly (10Q) or annual (10K) reports that public companies are required to file to the SEC — for this project is taken from a Kaggle dataset download[^1]. SEC filing data is notoriously difficult to parse since the XBRL formatting of its filings is clunky and hard to navigate. This dataset extracted key financial information from company 10-Q and 10-K filings of public companies and merged it with stock price history data from Yahoo Finance. The financial information comes from the SEC's bulk data downloads (company facts and submission history) and includes information about assets, debt, taxes and other data points in units of either dollars or stock shares that could give indications of a company's financial health/profile. The Yahoo Finance data is represented in price variance columns which are changes in price over 5, 10, 20, 30 and 60 day periods calculated from the day the form was filed. The data is in 4,624 thousand folders, one for each company.

[^1]: chartma. (2022). SEC Company Facts - All 10-Q & 10-K financial data [Data set](https://www.kaggle.com/datasets/chad116/sec-company-facts-all-10q-10k-financial-data/discussion). Kaggle. https://doi.org/10.34740/KAGGLE/DS/2292262

### B) ESG data

There are several organizations known for their ESG ratings, including MSCI ([Morgan Stanley Capital International](https://www.msci.com/documents/1296102/21901542/ESG-Ratings-Methodology-Exec-Summary.pdf)), Sustainalytics, Refinitiv, S&P Global and others. MSCI ratings seem to be the most common, so the other dataset I'm using in this project is a [Kaggle download](https://www.kaggle.com/datasets/debashish311601/esg-scores-and-ratings) of numeric MSCI ratings and breakdowns of E/S/G updated annually in June. This dataset was created in September and thus reflects the most recent MSCI ratings. It contains data from 15,000 companies, but I will filter it to include only data from public companies in the US. It contains columns of company name, ticker, sector, subsector, overall ESG score (in both categorical and numeric form) and the score's relevant components.˜

# Getting and preparing the data

The Kaggle financial dataset has a different CSV file First, I'm creating a list of csv filenames so I can read them into this notebook and eventually merge them into one dataset. There are 4,624 files — supposedly one for each publicly listed company in the U.S. [what is the stated number of publicly listed companies explain that it's like maybe 8000 but definition of company is different and talk about trusts/bdos/indexes/etfs]

In [35]:
import pandas as pd # HIDE THIS LATER
import numpy as np
import json
import time
import os
import collections
from dask import dataframe as df1
import matplotlib.pyplot as plt
import math
import re
# !pip install dask
# ! pip uninstall nbextension

To start, we'll look at one of these datafiles, the one for ticker SANM DISCUSS THIS A BIT
explain the columns a bit

In [161]:
one_file = pd.read_csv("/Users/phoebeliu/Downloads/SEC_data_kaggle/sec_data_made_useable_w_pricevar/sec_data_made_useable_w_pricevar/CIK0000897723_SANM.csv")
np.shape(one_file)

(51, 429)

In [162]:
one_file.head(10) 

,end,filed,form,Name,Ticker,CIK,Currency,PriceVar5,PriceVar10,PriceVar20,...,UnrecognizedTaxBenefitsResultingInNetOperatingLossCarryforward,UnrecognizedTaxBenefitsThatWouldImpactEffectiveTaxRate,ValuationAllowanceDeferredTaxAssetChangeInAmount,ValuationAllowancesAndReservesBalance,ValuationAllowancesAndReservesChargedToCostAndExpense,ValuationAllowancesAndReservesDeductions,WeightedAverageNumberDilutedSharesOutstandingAdjustment,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic,GainLossOnDispositionOfAssets1
0,2008-09-27,2008-11-24,10-K,Sanmina Corporation,SANM,CIK0000897723,USD,-0.020833,0.041667,0.041667,...,NaN,NaN,NaN,13422000.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-10-03,2009-12-01,10-K,Sanmina Corporation,SANM,CIK0000897723,USD,0.171393,0.186190,0.398274,...,NaN,NaN,NaN,16752000.0,3571000.0,-241000.0,NaN,82528000.0,82528000.0,NaN
2,2010-01-02,2010-02-05,10-Q,Sanmina Corporation,SANM,CIK0000897723,USD,0.117231,0.291109,0.376082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80575000.0,78615000.0,NaN
3,2010-04-03,2010-04-30,10-Q,Sanmina Corporation,SANM,CIK0000897723,USD,-0.097246,-0.048342,-0.109050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82782000.0,79001000.0,NaN
4,2010-07-03,2010-08-05,10-Q,Sanmina Corporation,SANM,CIK0000897723,USD,-0.128878,-0.149562,-0.229912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83693000.0,79544000.0,NaN
5,2010-10-02,2010-11-24,10-K,Sanmina Corporation,SANM,CIK0000897723,USD,-0.011680,0.020665,0.049416,...,NaN,NaN,NaN,16752000.0,-1187000.0,-1028000.0,NaN,82477000.0,79195000.0,NaN
6,2011-01-01,2011-02-08,10-Q,Sanmina Corporation,SANM,CIK0000897723,USD,-0.018349,-0.038532,-0.075229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82825000.0,79846000.0,NaN
7,2011-04-02,2011-04-26,10-Q,Sanmina Corporation,SANM,CIK0000897723,USD,0.000000,-0.035863,-0.123436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83940000.0,80242000.0,NaN
8,2011-07-02,2011-08-01,10-Q,Sanmina Corporation,SANM,CIK0000897723,USD,-0.141718,-0.263065,-0.365810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83141000.0,80579000.0,NaN
9,2011-10-01,2011-11-22,10-K,Sanmina Corporation,SANM,CIK0000897723,USD,-0.021879,0.145431,0.177606,...,NaN,NaN,14024000.0,14537000.0,-1187000.0,-1028000.0,NaN,83158000.0,80345000.0,NaN


In [172]:
# DESCRIPTION OF ABOVE?
# Look at dates/unique dates 
len(set(one_file["filed"])) # 51 dates 
print(one_file["filed"][0]) # min date [CLEAN THIS LATER]
print(one_file["filed"][50])
# also note how many are 10-k vs 10-q - write a sentence about this later
print(one_file["form"].value_counts())

2008-11-24
2022-02-02
10-Q    37
10-K    14
Name: form, dtype: int64


In [ ]:
os.chdir("/Users/phoebeliu/Downloads/SEC_data_kaggle/sec_data_made_useable_w_pricevar/sec_data_made_useable_w_pricevar")
csv_files = os.listdir()

In [ ]:
df = pd.concat(map(pd.read_csv, csv_files), ignore_index=True)
np.shape(df)

This dataset is of dimension 170497 x 11690 — one row for each filing (10-K or 10-Q a company made) and one column for each piece of "financial data." There's an unreasonably high number of columns because the names of certain financial markers changed over the years and depending on the type of company and type of form filed.

The next cell saves the file to my local drive so I can access it more quickly in the future.

In [ ]:
df.to_csv("/Users/phoebeliu/Downloads/SEC_Kaggle_Big_Test2.csv")

Now that I have the data saved on my computer, I can get it by reading it into the notebook. Because it's a big (2GB) file, reading it normally took too long (many hours). I'm using dask to speed up the process by splitting up the data into multiple dataframes so the reading can happen on multiple data "slices" in parallel and thus exponentially faster. The issue with dask is that there's a lot of slicing etc. going on, so it's almost impossible to index — so once we filter out some of this data, we will convert it back to a normal pandas dataframe.

In [2]:
s_time_dask = time.time()
dask_all = df1.read_csv('/Users/phoebeliu/Downloads/SEC_Kaggle_Big_Test.csv', sample=1000000)
e_time_dask = time.time()
print("Read with dask: ", (e_time_dask-s_time_dask), "seconds")
dask_all.head(10)

Read with dask:  0.7340621948242188 seconds


,Unnamed: 0,end,filed,form,Name,Ticker,CIK,Currency,PriceVar5,PriceVar10,...,ContractsInForceSubjectToParticipationThroughReinsuranceValue,IncomeTaxExpenseBenefitContinuingOperationsAdjustmentOfDeferredTaxAssetLiability.1,OtherComprehensiveIncomeLossFinancialLiabilityFairValueOptionBeforeTaxAfterReclassificationAdjustmentAttributableToParent,IncomeTaxHolidayIncomeTaxBenefitsPerShare.1,RetrospectiveEffectOfChangeInAccountingPrincipleTabularPresentationByFinancialStatementLineItemAmountOfApplicationAdjustment,AssumptionForFairValueOnSecuritizationDateOfInterestsContinuedToBeHeldByTransferorServicingAssetsOrLiabilitiesPrepaymentSpeed,EffectOfChangeInCashFlowProjectionEstimate,EntertainmentLicenseAgreementForProgramMaterialCapitalizedCost,FilmMonetizedOnItsOwnAndFilmMonetizedInFilmGroupAmortizationExpense,ProceedsFromDivestitureOfInterestInJointVenture.1
0,0,2011-04-30,2011-06-16,10-Q,"POINT OF CARE NANO-TECHNOLOGY, INC.",PCNT,CIK0001504239,USD,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2011-07-31,2011-11-10,10-K,"POINT OF CARE NANO-TECHNOLOGY, INC.",PCNT,CIK0001504239,USD,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2011-10-31,2011-12-20,10-Q,"POINT OF CARE NANO-TECHNOLOGY, INC.",PCNT,CIK0001504239,USD,-0.048387,-0.048387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2012-01-31,2012-03-21,10-Q,"POINT OF CARE NANO-TECHNOLOGY, INC.",PCNT,CIK0001504239,USD,0.000000,0.704546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2012-04-30,2012-06-14,10-Q,"POINT OF CARE NANO-TECHNOLOGY, INC.",PCNT,CIK0001504239,USD,0.000000,-0.180328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,2012-07-31,2012-11-13,10-K,"POINT OF CARE NANO-TECHNOLOGY, INC.",PCNT,CIK0001504239,USD,-0.166667,-0.166667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2012-10-31,2012-12-17,10-Q,"POINT OF CARE NANO-TECHNOLOGY, INC.",PCNT,CIK0001504239,USD,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,2013-01-31,2013-03-15,10-Q,"POINT OF CARE NANO-TECHNOLOGY, INC.",PCNT,CIK0001504239,USD,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,2013-04-30,2013-06-14,10-Q,"POINT OF CARE NANO-TECHNOLOGY, INC.",PCNT,CIK0001504239,USD,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,2013-07-31,2013-10-29,10-K,"POINT OF CARE NANO-TECHNOLOGY, INC.",PCNT,CIK0001504239,USD,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filtering financial data

This is a big dataset. Since MSCI ESG ratings are annual, let's first look at 10-K (annual report) data. 

In [3]:
dask_10ks = dask_all[dask_all["form"]=="10-K"]

In [ ]:
# saving the file to my disk so I can load it again later
# dask_10ks.to_csv("/Users/phoebeliu/Downloads/SEC_Kaggle_10Ks.csv")

In [ ]:
dask_10ks.head(5) # it checks out that all these are 10-Ks

Now, since we only have one year of MSCI ratings (historical data is not currently publicly accessible), we will get 10-Ks from just this past year (one per company). First, let's get everything in 2021 and after:

In [4]:
recent_10ks = dask_10ks[dask_10ks["filed"] > "2020-12-31"]

In [ ]:
recent_10ks.head(5) # showing that recent_10ks does what it is supposed to

Now, let's organize this to make sure there's only one report per company, and that it is the most recent one. 

In [5]:
tickerlist = dask_10ks["Ticker"].compute()
nonduplicates = list(set(tickerlist))
# tickerlist2 = recent_10ks["Ticker"].compute() # try to run this in the am -- it doesn't run, need to figure out why

In [ ]:
print([item for item, count in collections.Counter(tickerlist).items() if count > 2]) # come back to this too

## Trying to sort out discrepancies in number of public companies/filings

write a more elegant descriptor here rather than having messy comments

In [7]:
nonduplicates = pd.DataFrame(nonduplicates)
nonduplicates.to_csv("/Users/phoebeliu/Downloads/SEC_Kaggle_Tickers2.csv") #4588 tickers, we will see how many of them
# are reflected in the ratings data, saved this to a file that's now on my machine # why is this not 4624? 
# im convinced that tickers aren't completely unique (different exchange, merger, etc.)
# also note that the SEC data without pricevar has 5140 items HMMM what does Yahoo finance list vs. not list
len(nonduplicates)

4588

In [56]:
ciks = dask_10ks["CIK"]
nondup_ciks = list(set(ciks))

This is from when I pulled a list of CIKs from the SEC website itself by looking at everything that was filed. It has 7868 items, which is a lot more than what the Kaggle dataset has. What's up with that?

In [42]:
all_ciks = pd.read_csv("/Users/phoebeliu/Downloads/All_Tickers_Researched.csv")

In [15]:
diff = set(all_ciks) ^ set(nondup_ciks) # this is big bc not in the same format ... let's make them in the same format

In [57]:
nondup_ciks = [re.sub("CIK", "", elem) for elem in nondup_ciks]
nondup_ciks = [int(x) for x in nondup_ciks]

In [96]:
nondup_ciks.sort()
all_ciks = np.array(all_ciks)
set_dif = set(all_ciks).symmetric_difference(set(nondup_ciks))
difference = list(set_dif)
difference.sort()
len(difference) # 3671 ones that are different 
# difference[:50] # looking at what these are ????
# difference[3500:]

3671

Looked up a few of these, and they are things like 
STRATS(SM) TRUST FOR DOMINION RESOURCES, INC. SECURITIES, SERIES 2005-6
Rand Corporation (externally managed business dev company)
Spire alabama natural gas company
alabama power company
howmet aerospace
power and light

Lots of trusts, spacs, holding companies, etc. and most of them are "smaller reporting companies" which means that by the SEC's standards they don't have to report ESG risks so they aren't relevant anyways -- Looked them up in the SEC company facts API and didn't get results

So i think we're fine here
but are there companies in the first set not in the second?

In [100]:
print(len(set(all_ciks).difference(nondup_ciks)))
print(len(set(nondup_ciks).difference(all_ciks)))

# In other words, set(list_2).difference(list_1) returns a new set
# that contains the items in list_2 that are not in list_1.

3368
303


There are 3368 of these 'fake companies,' but there are also about 300 that are in the company facts data but not in the 10-K reports I found on the SEC's site. However, the SEC's site list was from only the past 1.5 years — possible that these companies exited (i.e. got acquired or are no longer public etc). Let's look at a few of them:

In [106]:
set(nondup_ciks).difference(all_ciks) #319458 might be a problem 
# same with 50292 and 91668
# might need to go through and check all of these because i might have missed a handful

{12659,
 22872,
 27367,
 30831,
 39020,
 50292,
 51548,
 91668,
 93205,
 100716,
 319458,
 352998,
 703699,
 704366,
 715788,
 716688,
 721693,
 725929,
 726293,
 727346,
 733337,
 758938,
 768710,
 784539,
 786947,
 811222,
 814586,
 819689,
 831355,
 834285,
 837852,
 842013,
 844538,
 862651,
 869531,
 883943,
 885074,
 896985,
 922330,
 925173,
 927720,
 930775,
 934796,
 946822,
 1001463,
 1002135,
 1009759,
 1016178,
 1023298,
 1025953,
 1030916,
 1031233,
 1040850,
 1054274,
 1058330,
 1058828,
 1059784,
 1065598,
 1069680,
 1075857,
 1084370,
 1084384,
 1085243,
 1085596,
 1089061,
 1089531,
 1091883,
 1092802,
 1093728,
 1094738,
 1096768,
 1099668,
 1104265,
 1106848,
 1108046,
 1114936,
 1117228,
 1126162,
 1133798,
 1141788,
 1142790,
 1156784,
 1157817,
 1159275,
 1172102,
 1178377,
 1191334,
 1223533,
 1227654,
 1239188,
 1267612,
 1270200,
 1281774,
 1284450,
 1284452,
 1287900,
 1288770,
 1295961,
 1300867,
 1301838,
 1302084,
 1310291,
 1313938,
 1318268,
 1318641,
 13

## Cutting down on the number of columns in financial data

11,000 columns is too many. First, let's drop all the columns that have no entries (probably old or 10-Q specific things). Our goal is to get this data into a manageable size so we can convert it back to a non-dask dataframe to analyze it more conveniently. This gets us to 7408, which is ... still a lot.

In [107]:
recent_10ks_withoutna = recent_10ks.loc[:, ~recent_10ks.isna().all().compute()]

KeyboardInterrupt: 

Now, let's set a threshold to cut down the number of columns even more. I might come back later and edit this, but for now I'm setting an arbitrary threshold of 3000, or about half the companies (since there are 4588 companies represented and each one should have 1-2 10-K reports filed since 2021 began). So these columns would still be majority null/NA. This results in 81 columns, which is much more manageable.

In [ ]:
THRESHOLD = 3000
recent_10ks_fewer_na = recent_10ks_withoutna.loc[:, recent_10ks_withoutna.isna().sum().compute() < THRESHOLD] 

In [ ]:
recent_10ks_fewer_na.head(5)

Now, we can convert this back to a regular dataframe. We will use this dataset later to merge with the ESG rating data.

In [ ]:
final_10ks = recent_10ks_fewer_na.compute()

In [ ]:
# save this dataset to local drive so I can access it quickly later if I want to 
# final_10ks.to_csv("/Users/phoebeliu/Downloads/Final_10k_document.csv") 

## Getting ESG scores data

Now, I'm uploading the file of ESG scores (MSCI index) I downloaded from Kaggle and filtering out non-public and non-US companies --> 4349 unique tickers. [do more exploration of this data and write about it]

In [108]:
scores = pd.read_csv("/Users/phoebeliu/Downloads/sustainability_scores.csv") # MSCI scores
public_scores = scores[scores["Ticker"] == scores["Ticker"]] # getting rid of non public companies
public_us_scores = public_scores[public_scores["Country"]=="US"] # getting US only
SEC_tickers = final_10ks["Ticker"] #4349 unique tickers
intersect_ticker = list(set(SEC_tickers) & set(public_us_scores["Ticker"])) # has 2095 companies
np.shape(intersect_ticker)

NameError: name 'final_10ks' is not defined

In [109]:
scores.head(10)

,Company Name,Ticker,Country,Sector,Subsector,Overall ESG RATING,Overall ESG SCORE,Environmental SCORE,Social SCORE,Governance SCORE
0,0921706 BC LTD,NaN,CA,Telecommunication Services,Wireless Telecommunication Services,A,6.6,10.0,7.4,2.7
1,1 MADISON OFFICE FEE LLC,NaN,US,Real Estate Management & Services,Office REITs,BBB,4.7,8.5,5.8,2.6
2,1&1 AG,1U1,DE,Telecommunication Services,Wireless Telecommunication Services,B,2.0,7.8,3.3,4.4
3,"1-800-FLOWERSCOM, INC",FLWS,US,Retail - Consumer Discretionary,Internet & Direct Marketing Retail,BBB,5.6,1.2,6.7,3.8
4,"1000 AUAHI, LLC",NaN,US,Real Estate Development & Diversified Activities,Real Estate Development,A,6.8,6.1,4.1,6.9
5,1011778 BC UNLIMITED LIABILITY COMPANY,NaN,CA,Restaurants,Restaurants,BB,3.9,5.3,3.6,4.4
6,10X Genomics Inc,TXG,US,Health Care Equipment & Supplies,Life Sciences Tools & Services,BB,3.4,6.1,3.9,5.1
7,"1334 YORK, LLC",NaN,US,Diversified Consumer Services,NaN,BBB,5.5,6.7,5.4,3.6
8,"180 MEDICAL, INC",NaN,US,Health Care Equipment & Supplies,Health Care Supplies,AA,8.4,10.0,5.2,7.1
9,"1LIFE HEALTHCARE, INC",ONEM,US,Health Care Providers & Services,Health Care Services,A,6.3,6.3,6.0,4.2


In [157]:
len(set(nonduplicates[0]).difference(public_us_scores["Ticker"]))
# There are 4588 unique nonduplicates (from financial data)
# There are 2833 tickers
# Which means that there difference between these sets should be 1755
# Maybe I should make my own ESG scores dataset
print(set(nonduplicates[0]).difference(public_us_scores["Ticker"]))

{'MSN', 'MMMW', 'FGCO', 'DIAH', 'FSRL', 'VALU', 'MLSS', 'KBRS', 'DESTQ', 'TMBR', 'SFBC', 'CDXI', 'PSTI', 'OTIC', 'AKUS', 'HHS', 'MYMD', 'DRIO', 'UAMY', 'BTDG', 'SIGO', 'LDSN', 'GLG', 'NBY', 'MYO', 'FTLF', 'GKIN', 'AVGR', 'CVSI', 'GFMH', 'AVDL', 'PRKA', 'PLYA', 'TCRR', 'CWRK', 'AYRO', 'LMST', 'EVFM', 'SREV', 'STX', 'ADMG', 'BANT', 'SEVN', 'GLBZ', 'NVEC', 'CGLO', 'BIOL', 'MUX', 'GLGI', 'PTPI', 'PULM', 'HSDT', 'RAYT', 'CZWI', 'SONM', 'TBPH', 'RETC', 'DLTI', 'USAQ', 'SUNL', 'CYAP', 'EZRG', 'TCRI', 'NANX', 'POLA', 'TWIN', 'KEQU', 'FVTI', 'LBAO', 'OCLN', 'PDEX', 'CDJM', 'WVFC', 'WNEB', 'SELF', 'SPRO', 'CPRI', 'GLRE', 'THCT', 'EPIX', 'ATXI', 'SODI', 'FLXT', 'SCIA', 'PLYN', 'NXTP', 'VIZC', 'WINT', 'TREC', 'ALT', 'MTNB', 'BDRY', 'VRVR', 'FUST', 'VFF', 'SNFCA', 'CHCR', 'EDUC', 'EMYB', 'FZMD', 'MPB', 'PLSI', 'UUU', 'RVRF', 'MDVL', 'RIVX', 'KBLB', 'ELMD', 'ATMS', 'NTOX', 'PRCX', 'INZY', 'FSTR', 'GIFI', 'HMNF', 'QBIO', 'UNEX', 'CIIT', 'TAYD', 'CTGO', 'PBLA', 'REPX', 'ARRT', 'ASDN', 'AWON', 'ADES', 

HMMM. Most of these companies have never filed anything with the SEC and several don't even seem to trade under this ticker. MSCI rates "equity and fixed income securities, loans, mutual funds, ETFs and countries" which don't file 10-Ks with the SEC.

In [156]:
print(len(set(public_us_scores["Ticker"]).difference(nonduplicates[0]))) # there are 737 the other way around hmmm
# IBRD is a bank -- companies in scores that are not in nonduplicates
print(set(public_us_scores["Ticker"]).difference(nonduplicates[0]))

737
{'KSU*', 'IBRD', 'FXLV', 'IONMN', 'PACW', 'IIVI', 'AVAH', 'MAPS', 'FIGS', 'RKERR', 'JW.A', 'D', 'SLVM', 'OWE', 'HVT', 'WEY', 'MT', 'UWMC', 'IPLXX', 'ENI', 'FMRCO', 'HTGC', 'BRK', 'TAKDA', 'STI', 'ETRN', 'OWL', 'AAM PR A', 'TRPTC', 'TRTX', 'BOLT', 'CRAWG', 'CLDR', 'MLNK', 'CWEN.A', 'CRC', 'DKNG', 'AGNC', 'HALC', 'BFS', 'NAVI', 'BNGO', 'ZIP', 'DAST', 'DBAG', 'DHC', 'NMUIC', 'GIL', 'SPG', 'DFRG', 'TOWN', 'STZ', 'NECEL', 'NESN', 'RABO', 'BPYU', 'STEKR', 'BAYG', 'NONAH', 'NEE', 'FRTA', 'EQC', 'DLR', 'EVRI', 'WTFC', '2148', 'MCRO', 'BID', 'RYAN', 'GPOR', 'SF', 'RC', 'TRMPR', 'TSVT', 'FFIE', 'XMTR', 'UAA', 'CHNG', 'SGHT', 'IVR', 'HL', 'LDOF', 'NICRL', 'CIMXP', 'COWN', 'JOSPH', 'SYNTS', 'ADS', 'TKOMH', 'WFC', 'HUMA', 'GRBK', 'PRVA', 'NOBLT', 'JPM', 'NYMT', 'PECO', 'PAY', 'SGMS', 'MMGFB', 'CCSI', 'ORGN', 'BLS*', 'SHEHM', 'NTNX', 'AVPT', 'XRXHD', 'TWO', 'ROLL', 'ZGNX', 'PPWLM', 'KNBE', 'PNFP', 'DCPX', 'DCP', 'RB', 'XENT', 'GPMT', 'APU', 'NSA', 'PK', 'CANO', 'GSK', 'AQN', 'PCG', 'ZION', 'EPAY

In [149]:
len(set(public_us_scores["Ticker"]))

2833

# Exploring the data

What columns does this list have?

In [ ]:
# plt.hist((final_10ks["Assets"]), bins = 50) # this might be erroneous code
list(final_10ks.columns)
plt.hist(np.log(final_10ks["Assets"]+1), bins=25)

In [ ]:
# need to get the intersection of these and then filter the df by that
filtered_10ks = final_10ks.loc[final_10ks['Ticker'].isin(intersect_ticker)] 
# 3768 rows * 81 columns
filtered_scores = scores.loc[scores['Ticker'].isin(intersect_ticker)] 
# is this enough data to work with? maybe i should go through and get more years

In [ ]:
# now want to merge this df with the ratings df, but we first need to only get the most 
# recent 10k report -- the most recent value in this dataset was a report filed on march 18
# so let's filter this further to only include things from march 19 2021 to march 18 2022
# and see if the size matches that of the filtered_scores (where there is only one per company)

# sorting the data by date (descending)
filtered_10ks = filtered_10ks.sort_values(by=['filed'],ascending=False)

filtered_10ks_2 = filtered_10ks[filtered_10ks["filed"] > "2021-03-19"]
np.shape(filtered_10ks_2) # now has 2140 rows but filtered_scores has 2550
# did some companies ... not submit a 10K??? this is sill more than the 2095 unique
# entries in the scores

np.shape(filtered_scores["Ticker"]) #2550 entries
len(set(filtered_scores["Ticker"])) # 2095 unique entries

# realizing that the duplicates in the scores come mostly from M&A things and have the same
# ratings, let's get rid of those 
filtered_scores_2 = filtered_scores.drop_duplicates(subset='Ticker', keep='first')
# ok this is the 2095 x 10 thing i want 
filtered_10ks_3 = filtered_10ks_2.drop_duplicates(subset='Ticker', keep='first')
# ok now i have 2078 ... maybe it's bc some companies didn't submit 10ks
# we can work with this

In [ ]:
merged = pd.merge(filtered_10ks_3, filtered_scores_2, on='Ticker') # this seems to work
# now we have some duplicate columns we can take out ???
# first of all ... what is this first column 
# second of all: currency column is prob stupid bc they're all USD
# can remove company name, country
merged_final = merged.drop(["Country","Company Name","Currency"], axis=1)

In [ ]:
merged_final # this seems good!?!?!?!?
# save it to local inccase 
merged_final.to_csv("/Users/phoebeliu/Downloads/SDS_Thesis_Merged_Data.csv")

In [ ]:
np.shape(merged_final)

In [ ]:
# looking at relationship between scores and assets??? 
plt.scatter(np.log(merged_final["Assets"]+1), merged_final["Overall ESG SCORE"], c ="blue")

In [ ]:
# is it time to make a corrplot lol? or at least a matrix
corr = merged_final.corr()
pd.set_option('max_rows', None)
print(corr["Overall ESG SCORE"])
pd.reset_option('max_rows') # look at assets, common stock shares outstanding,
# income tax expense benefit, // sort this list and look at it that way
# need to break this down and look at categorical variables 

In [ ]:
# lin reg?
from sklearn.linear_model import LinearRegression
X = 
linreg = LinearRegression().fit(X, y)

# should p